In [86]:
from pandas import read_excel, DataFrame
import pandas as pd

In [87]:
df = read_excel("nueva_base_de_datos_limpia.xlsx")
# Borra columnas superfluas
df = df[["NOMBRE", "MATRICULA", "CARRERA", "CREDITOS", "PLANTEL"]]
df.head()

,NOMBRE,MATRICULA,CARRERA,CREDITOS,PLANTEL
0,Alejandra Lopez Millan,13-001-1254,Arte y Patrimonio Cultural,123,Casa Libertad
1,Alejandro De la Rea ledesma,12-011-1537,Nutrición y Salud,335,San lorenzo
2,Alejandro Hernández Hernández,13-011-0026,Creación Literaria,230,Centro Historico
3,Alejandro Juárez Arellano,08-011-1000,Creación Literaria,262,San Lorenzo Tezonco
4,Alerick Santana Quiroz,12-011-2654,Protección Civil y Gestión de Riesgos,65,Casa Libertad


In [88]:
# Limpieza de los nombres de planteles
# value, como diccionario, sólo hace reemplazos en las columnas dadas como claves

#df.replace(to_replace=".*ist.*", value="Centro Histórico", regex=True, inplace=True) # NO!, o habrá carrera llamada 'Centro histórico'
df.replace(to_replace=".*ist.*", value={'PLANTEL':"Centro Histórico"}, regex=True, inplace=True)
df.replace(to_replace=".*ibertad.*", value={'PLANTEL':"Casa Libertad"}, regex=True, inplace=True)
df.replace(to_replace=".*oren.*", value={'PLANTEL':"San Lorenzo Tezonco"}, regex=True, inplace=True)
df.replace(to_replace=".*pec", value={'PLANTEL':"Cuautepec"}, regex=True, inplace=True)
df.replace(to_replace=".*alle", value={'PLANTEL':"Del Valle"}, regex=True, inplace=True)
df['PLANTEL'].unique()

array(['Casa Libertad', 'San Lorenzo Tezonco', 'Centro Histórico',
       'Del Valle', 'Cuautepec', nan], dtype=object)

In [96]:
# los nan son valores float, 
import numpy as np
import math
#df[df['PLANTEL'].map(type)==float]['PLANTEL']
#df[df['PLANTEL'].map(type)!=np.str]
#type(df.iloc[42]['PLANTEL'])

In [97]:
# fillna puede hacerse cargo de los NaN, nóte el value tipo dict
df = df.fillna({'PLANTEL':'Cuautepec'})
df['PLANTEL'].unique()

array(['Casa Libertad', 'San Lorenzo Tezonco', 'Centro Histórico',
       'Del Valle', 'Cuautepec'], dtype=object)

# Preguntas
1. Ordenar alumnos por créditos, cohorte
2. Carrera con más avance (sum(créditos por materia)
3. Ordenar las cohortes (alumnos que entraron el mismo año), de mayor a menor número de alumnos
4. Ordenar carreras por número de alumnos
5. Porcentaje de alumnos de cada carrera en cada plantel
6. Crecimiento de matrícula por plantel
7. Relación de migración de alumnos de plantel X al plantel Y y viceverza
8. Ordenar alumnos por el cociente de créditos que tiene, entre los créditos que debería tener (%deseado) al semestre actual dependiendo de cuando entró (para detectar alumnos atrasados)
9. Graficar para cada semestre el promedio de créditos de esa cohorte, multiplicado por el factor de (1 - %deseado)

In [100]:
df['AÑO']=df['MATRICULA'].map(lambda s: 2000 + int(s.split("-")[0]))
df.head()

,NOMBRE,MATRICULA,CARRERA,CREDITOS,PLANTEL,AÑO
0,Alejandra Lopez Millan,13-001-1254,Arte y Patrimonio Cultural,123,Casa Libertad,2013
1,Alejandro De la Rea ledesma,12-011-1537,Nutrición y Salud,335,San Lorenzo Tezonco,2012
2,Alejandro Hernández Hernández,13-011-0026,Creación Literaria,230,Centro Histórico,2013
3,Alejandro Juárez Arellano,08-011-1000,Creación Literaria,262,San Lorenzo Tezonco,2008
4,Alerick Santana Quiroz,12-011-2654,Protección Civil y Gestión de Riesgos,65,Casa Libertad,2012


In [123]:
# otra versión
#df_creditos = df.sort_values("CREDITOS", ascending=False)
#es necesario pivot?
#df2 = df_creditos.pivot_table('MATRICULA', index=['AÑO', 'CREDITOS', 'NOMBRE'], aggfunc=np.count_nonzero)

df2 = DataFrame(df, columns=['AÑO','CREDITOS','NOMBRE'])
respuesta1 = df2.sort_values(['AÑO','CREDITOS','NOMBRE'], ascending=False)
respuesta1.head()

,AÑO,CREDITOS,NOMBRE
105,2015,136,Victor Ortega Pèrez
79,2015,60,Maricruz MAteo Barrios
7,2015,40,Arely Mendoza Camacho
109,2015,20,Wendy Rodriguez Ochoa
36,2014,259,Fernando Esquibel Montero


In [138]:
import numpy
carreras = df.pivot_table('CREDITOS', index="CARRERA", aggfunc=numpy.sum)
respuesta2 = carreras.sort_values(ascending=False)
respuesta2

CARRERA
Ingeniería de Software                              5090
Ingenieria en Sistemas Electrónicos Industriales    3824
Creación Literaria                                  2768
Nutrición y Salud                                   2541
Promoción de la Salud                               2319
Arte y Patrimonio Cultural                          2211
Derecho                                             1500
Comunicación y Cultura                              1255
Protección Civil y Gestión de Riesgos               1090
Ingenieria en Transporte Urbano                      770
História y Sociedad Contemporánea                    752
Filosofía e Historia de las Ideas                    474
Ciencias politicas                                   420
Ciencias Sociales                                    412
Ciencia Política y Administración Urbana             384
Ciencias Ambientales y Cambio Climático              215
Ingenieria en Sistemas Energéticos                   210
Modelación Matemática  

In [142]:
mataño = df[['AÑO', 'MATRICULA']]
mataño.head()

,AÑO,MATRICULA
0,2013,13-001-1254
1,2012,12-011-1537
2,2013,13-011-0026
3,2008,08-011-1000
4,2012,12-011-2654


In [145]:
df2 = mataño.pivot_table("MATRICULA", index='AÑO', aggfunc=numpy.count_nonzero)
respuesta3 = df2.sort_values(ascending=False)
respuesta3

AÑO
2012    39
2013    16
2011    16
2009    15
2010    14
2014    12
2015     4
2008     3
2007     3
Name: MATRICULA, dtype: int64

In [12]:
resultado_pregunta_3=pregunta_3_parcial.sort_values(ascending=False)
resultado_pregunta_3

MATRICULA
12    39
13    16
11    16
09    15
10    14
14    12
15     4
08     3
07     3
Name: MATRICULA, dtype: int64

In [13]:
p4df1 = df.pivot_table("MATRICULA", index="CARRERA", aggfunc=numpy.count_nonzero)

In [14]:
resultado_pregunta_4 = p4df1.sort_values(ascending=False)

In [15]:
resultado_pregunta_4

CARRERA
Ingeniería de Software                              23
Ingenieria en Sistemas Electrónicos Industriales    15
Arte y Patrimonio Cultural                          12
Promoción de la Salud                               12
Nutrición y Salud                                   11
Creación Literaria                                  10
Comunicación y Cultura                               7
Derecho                                              7
Protección Civil y Gestión de Riesgos                4
Ingenieria en Transporte Urbano                      4
Filosofía e Historia de las Ideas                    4
História y Sociedad Contemporánea                    3
Modelación Matemática                                2
Ciencias Sociales                                    2
Ciencias Ambientales y Cambio Climático              2
Ciencia Política y Administración Urbana             2
Ingenieria en Sistemas Energéticos                   1
Ciencias politicas                                   1
Na

In [16]:
df.pivot_table("MATRICULA", aggfunc=numpy.count_nonzero, index="CARRERA")

CARRERA
Arte y Patrimonio Cultural                          12
Ciencia Política y Administración Urbana             2
Ciencias Ambientales y Cambio Climático              2
Ciencias Sociales                                    2
Ciencias politicas                                   1
Comunicación y Cultura                               7
Creación Literaria                                  10
Derecho                                              7
Filosofía e Historia de las Ideas                    4
História y Sociedad Contemporánea                    3
Ingenieria en Sistemas Electrónicos Industriales    15
Ingenieria en Sistemas Energéticos                   1
Ingenieria en Transporte Urbano                      4
Ingeniería de Software                              23
Modelación Matemática                                2
Nutrición y Salud                                   11
Promoción de la Salud                               12
Protección Civil y Gestión de Riesgos                4
Na

In [23]:
cuentas = df.pivot_table('MATRICULA', index='CARRERA', columns='PLANTEL', aggfunc=numpy.count_nonzero)
cuentas

PLANTEL,Casa Libertad,Centro Histórico,Cuautepec,Del Valle,San Lorenzo Tezonco
CARRERA,,,,,
Arte y Patrimonio Cultural,4.0,NaN,1.0,4.0,3.0
Ciencia Política y Administración Urbana,1.0,NaN,NaN,1.0,NaN
Ciencias Ambientales y Cambio Climático,1.0,NaN,NaN,NaN,1.0
Ciencias Sociales,NaN,NaN,1.0,1.0,NaN
Ciencias politicas,NaN,NaN,1.0,NaN,NaN
Comunicación y Cultura,NaN,3.0,1.0,1.0,2.0
Creación Literaria,NaN,2.0,3.0,3.0,2.0
Derecho,NaN,2.0,5.0,NaN,NaN
Filosofía e Historia de las Ideas,2.0,NaN,1.0,NaN,1.0


In [24]:
# Los NaN resultantes son de que no hay estudiantes para esas carreras en esos planteles, rellenar con 0
cuentas=cuentas.fillna(0)

In [25]:
cuentas

PLANTEL,Casa Libertad,Centro Histórico,Cuautepec,Del Valle,San Lorenzo Tezonco
CARRERA,,,,,
Arte y Patrimonio Cultural,4.0,0.0,1.0,4.0,3.0
Ciencia Política y Administración Urbana,1.0,0.0,0.0,1.0,0.0
Ciencias Ambientales y Cambio Climático,1.0,0.0,0.0,0.0,1.0
Ciencias Sociales,0.0,0.0,1.0,1.0,0.0
Ciencias politicas,0.0,0.0,1.0,0.0,0.0
Comunicación y Cultura,0.0,3.0,1.0,1.0,2.0
Creación Literaria,0.0,2.0,3.0,3.0,2.0
Derecho,0.0,2.0,5.0,0.0,0.0
Filosofía e Historia de las Ideas,2.0,0.0,1.0,0.0,1.0


In [26]:
cuentas['número de estudiantes']=cuentas['Casa Libertad']+cuentas['Centro Histórico']+\
    cuentas['Cuautepec']+cuentas['Del Valle']+cuentas['San Lorenzo Tezonco']

In [27]:
cuentas

PLANTEL,Casa Libertad,Centro Histórico,Cuautepec,Del Valle,San Lorenzo Tezonco,número de estudiantes
CARRERA,,,,,,
Arte y Patrimonio Cultural,4.0,0.0,1.0,4.0,3.0,12.0
Ciencia Política y Administración Urbana,1.0,0.0,0.0,1.0,0.0,2.0
Ciencias Ambientales y Cambio Climático,1.0,0.0,0.0,0.0,1.0,2.0
Ciencias Sociales,0.0,0.0,1.0,1.0,0.0,2.0
Ciencias politicas,0.0,0.0,1.0,0.0,0.0,1.0
Comunicación y Cultura,0.0,3.0,1.0,1.0,2.0,7.0
Creación Literaria,0.0,2.0,3.0,3.0,2.0,10.0
Derecho,0.0,2.0,5.0,0.0,0.0,7.0
Filosofía e Historia de las Ideas,2.0,0.0,1.0,0.0,1.0,4.0


In [28]:
cuentas['pCasa Libertad']=cuentas['Casa Libertad']/cuentas['número de estudiantes']*100
cuentas['pCuautepec']=cuentas['Cuautepec']/cuentas['número de estudiantes']*100
cuentas['pCentro Histórico']=cuentas['Centro Histórico']/cuentas['número de estudiantes']*100
cuentas['pDel Valle']=cuentas['Del Valle']/cuentas['número de estudiantes']*100
cuentas['pSan Lorenzo Tezonco']=cuentas['San Lorenzo Tezonco']/cuentas['número de estudiantes']*100
respuesta5 = pd.DataFrame(cuentas, columns = ['pCasa Libertad','pCuautepec','pCentro Histórico',
                                                    'pDel Valle', 'pSan Lorenzo Tezonco', 'número de estudiantes'])
respuesta5

,pCasa Libertad,pCuautepec,pCentro Histórico,pDel Valle,pSan Lorenzo Tezonco,número de estudiantes
CARRERA,,,,,,
Arte y Patrimonio Cultural,33.333333,8.333333,0.000000,33.333333,25.000000,12.0
Ciencia Política y Administración Urbana,50.000000,0.000000,0.000000,50.000000,0.000000,2.0
Ciencias Ambientales y Cambio Climático,50.000000,0.000000,0.000000,0.000000,50.000000,2.0
Ciencias Sociales,0.000000,50.000000,0.000000,50.000000,0.000000,2.0
Ciencias politicas,0.000000,100.000000,0.000000,0.000000,0.000000,1.0
Comunicación y Cultura,0.000000,14.285714,42.857143,14.285714,28.571429,7.0
Creación Literaria,0.000000,30.000000,20.000000,30.000000,20.000000,10.0
Derecho,0.000000,71.428571,28.571429,0.000000,0.000000,7.0
Filosofía e Historia de las Ideas,50.000000,25.000000,0.000000,0.000000,25.000000,4.0


In [32]:
#Crecimiento de matrícula por año por plantel (primeros dos dígitos de MATRICULA es el año de entrada)
df.head()

,NOMBRE,MATRICULA,CARRERA,CREDITOS,PLANTEL
0,Alejandra Lopez Millan,13-001-1254,Arte y Patrimonio Cultural,123,Casa Libertad
1,Alejandro De la Rea ledesma,12-011-1537,Nutrición y Salud,335,San Lorenzo Tezonco
2,Alejandro Hernández Hernández,13-011-0026,Creación Literaria,230,Centro Histórico
3,Alejandro Juárez Arellano,08-011-1000,Creación Literaria,262,San Lorenzo Tezonco
4,Alerick Santana Quiroz,12-011-2654,Protección Civil y Gestión de Riesgos,65,Casa Libertad
